# Notebook Setup 

## Setup Drake on Google Colab (if necessary)

The following cell will install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.

On Google's Colaboratory, this will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours. Colab may ask you to "Reset all runtimes"; say no to save yourself the reinstall.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  urlretrieve("https://raw.githubusercontent.com/RobotLocomotion/drake/master/tools/install/colab/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version='20200805', build='nightly') 

In [ ]:
server_args = []
if 'google.colab' in sys.modules:
  !pip install pyngrok
  server_args = ['--ngrok_http_tunnel']

## Patches / hotfixes / WIP

The following cells will all disappear once I land them in drake / meshcat. -- Russ


In [ ]:
#import os
#if not os.path.isdir('/content/meshcat-python'):
#  !git clone https://github.com/RussTedrake/meshcat-python --branch colab --recursive
#sys.path = ['/content/meshcat-python/src'] + sys.path
#print(importlib.util.find_spec('meshcat').origin)

In [ ]:
import os
pydrake_path = os.path.dirname(importlib.util.find_spec('pydrake').origin)
#os.remove(f"{pydrake_path}/systems/jupyter_widgets.py")
urlretrieve("https://raw.githubusercontent.com/RussTedrake/drake/widget_systems/bindings/pydrake/systems/jupyter_widgets.py",
              f"{pydrake_path}/systems/jupyter_widgets.py");
#urlretrieve("https://raw.githubusercontent.com/RussTedrake/drake/meshcat_jupyter/bindings/pydrake/systems/meshcat_visualizer.py",
#              f"{pydrake_path}/systems/meshcat_visualizer.py");
urlretrieve("https://raw.githubusercontent.com/RussTedrake/drake/widget_systems/bindings/pydrake/systems/meshcat_visualizer.py",
              f"{pydrake_path}/systems/meshcat_visualizer.py");

In [ ]:
def open_window(url, name):
    if google.colab in sys.modules:
        # This has the benefit of not persisting between notebooks if the output 
        # is accidentally saved.
        google.colab.output.eval_js(
            f'window.open("{url}", name);', 
            ignore_result=True)
    else:
        from IPython.display import display, Javascript
        display(Javascript(f'window.open("{url}", name);'))

In [ ]:
import numpy as np
from IPython.display import display
from ipywidgets import Checkbox
from pydrake.manipulation.planner import DoDifferentialInverseKinematics
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.systems.framework import BasicVector, EventStatus, LeafSystem, PortDataType
from pydrake.systems.jupyter_widgets import update_widgets


# TODO(russt): Generalize this and move it to pydrake.manipulation.simple_ui.
class SchunkWsgTeleop(LeafSystem):
    """
    Adds a checkbox to open/close the Schunk WSG gripper.

    @system{ SchunkWsgTeleop,
             , # no input ports
             @output_port{position}
             @output_port{max_force} }
    """

    def __init__(self, open_position=0.107,
                 closed_position=0.002, force_limit=40,
                 update_period_sec=0.05):
        """"
        Args:
            update_period_sec: Specifies how often the window update() method
                             gets called.
            open_position:   Target position for the finger when open.
            closed_position: Target position for the gripper when closed.
            force_limit:     Force limit to send to Schunk WSG controller.
        """
        LeafSystem.__init__(self)
        self.DeclareVectorOutputPort("position", BasicVector(1),
                                     self.CalcPositionOutput)
        self.DeclareVectorOutputPort("force_limit", BasicVector(1),
                                     self.CalcForceLimitOutput)

        # Schedule widget updates:
        self.DeclarePeriodicPublish(update_period_sec, 0.0)

        self._checkbox = Checkbox(description="Close Gripper")
        display(self._checkbox)

        self._open_position = open_position
        self._closed_position = closed_position
        self._force_limit = force_limit

    def DoPublish(self, context, event):
        update_widgets()

    def CalcPositionOutput(self, context, output):
        if self._checkbox.value:
            # Push to joint limit specified in schunk_wsg_50.sdf.
            output.SetAtIndex(0, self._closed_position)
        else:
            # Closing to 0mm can smash the fingers together and keep applying
            # force even when no object is grasped.
            output.SetAtIndex(0, self._open_position)

    def CalcForceLimitOutput(self, context, output):
        output.SetAtIndex(0, self._force_limit)

# TODO(russt): Clean this up and move it to C++.
class DifferentialIK(LeafSystem):
    """
    A simple system that wraps calls to the DifferentialInverseKinematics API.
    It is highly recommended that the user calls SetPosition() once to
    initialize the initial position commands to match the initial
    configuration of the robot.

    @system{
      @input_port{X_WE_desired},
      @output_port{joint_position_desired}
    """
    def __init__(self, robot, frame_E, parameters, time_step):
        """
        @param robot is a reference to a MultibodyPlant.
        @param frame_E is a multibody::Frame on the robot.
        @param params is a DifferentialIKParams.
        @params time_step This system updates its state/outputs at discrete
                          periodic intervals defined with period @p time_step.
        """
        LeafSystem.__init__(self)
        self.robot = robot
        self.frame_E = frame_E
        self.parameters = parameters
        self.parameters.set_timestep(time_step)
        self.time_step = time_step
        # Note that this context is NOT the context of the DifferentialIK
        # system, but rather a context for the multibody plant that is used
        # to pass the configuration into the DifferentialInverseKinematics
        # methods.
        self.robot_context = robot.CreateDefaultContext()
        # Confirm that all velocities are zero (they will not be reset below).
        assert not self.robot.GetPositionsAndVelocities(
            self.robot_context)[-robot.num_velocities():].any()

        # Store the robot positions as state.
        self.DeclareDiscreteState(robot.num_positions())
        self.DeclarePeriodicDiscreteUpdate(time_step)

        # Desired pose of frame E in world frame.
        self.DeclareInputPort("rpy_xyz_desired",
                              PortDataType.kVectorValued, 6)

        # Provide the output as desired positions.
        self.DeclareVectorOutputPort("joint_position_desired", BasicVector(
            robot.num_positions()), self.CopyPositionOut)

    def SetPositions(self, context, q):
        context.get_mutable_discrete_state(0).SetFromVector(q)

    def ForwardKinematics(self, q):
        x = self.robot.GetMutablePositionsAndVelocities(
            self.robot_context)
        x[:self.robot.num_positions()] = q
        return self.robot.EvalBodyPoseInWorld(
            self.robot_context, self.frame_E.body())

    def CalcPoseError(self, X_WE_desired, q):
        pose = self.ForwardKinematics(q)
        err_vec = np.zeros(6)
        err_vec[-3:] = X_WE_desired.translation() - pose.translation()

        rot_err = AngleAxis(X_WE_desired.rotation()
                            * pose.rotation().transpose())
        err_vec[:3] = rot_err.axis() * rot_err.angle()

    def DoCalcDiscreteVariableUpdates(
            self, context, events, discrete_state):
        rpy_xyz_desired = self.EvalVectorInput(context, 0).get_value()
        X_WE_desired = RigidTransform(RollPitchYaw(rpy_xyz_desired[:3]),
                                      rpy_xyz_desired[-3:]).GetAsIsometry3()
        q_last = context.get_discrete_state_vector().get_value()

        x = self.robot.GetMutablePositionsAndVelocities(
            self.robot_context)
        x[:self.robot.num_positions()] = q_last
        result = DoDifferentialInverseKinematics(self.robot,
                                                 self.robot_context,
                                                 X_WE_desired, self.frame_E,
                                                 self.parameters)

        if (result.status != result.status.kSolutionFound):
            print("Differential IK could not find a solution.")
            discrete_state.get_mutable_vector().SetFromVector(q_last)
        else:
            discrete_state.get_mutable_vector().\
                SetFromVector(q_last + self.time_step*result.joint_velocities)

    def CopyPositionOut(self, context, output):
        output.SetFromVector(context.get_discrete_state_vector().get_value())


## Imports

In [ ]:
import numpy as np
from pydrake.all import (
    ConnectMeshcatVisualizer, DiagramBuilder, RigidTransform, RotationMatrix, 
    DifferentialInverseKinematicsParameters, Simulator
)
from pydrake.systems.jupyter_widgets import PoseSliders
from pydrake.examples.manipulation_station import ManipulationStation, CreateClutterClearingYcbObjectList

# Teleop Example (3D)

In this example, we assemble a diagram with all of the relevant subsystems (the manipulation station, the meshcat visualizer, and some systems that provide a minimal teleop interface and convert the teleop output from end-effector commands into joint commands.  We'll learn more about each of these components in the following chapters.


In [ ]:
builder = DiagramBuilder()

station = builder.AddSystem(ManipulationStation())

station.SetupClutterClearingStation()
ycb_objects = CreateClutterClearingYcbObjectList()
#for model_file, X_WObject in ycb_objects:
#    station.AddManipulandFromFile(model_file, X_WObject)
station.Finalize()

visualizer = ConnectMeshcatVisualizer(
    builder, 
    station.get_scene_graph(), 
    station.GetOutputPort("pose_bundle"),
    zmq_url="new",
    server_args=server_args)
#    jupyter_comms=True)

robot = station.get_controller_plant()
params = DifferentialInverseKinematicsParameters(robot.num_positions(),
                                                  robot.num_velocities())

time_step = 0.005
params.set_timestep(time_step)
# True velocity limits for the IIWA14 (in rad, rounded down to the first
# decimal)
iiwa14_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                  iiwa14_velocity_limits))
differential_ik = builder.AddSystem(DifferentialIK(
    robot, robot.GetFrameByName("iiwa_link_7"), params, time_step))
builder.Connect(differential_ik.GetOutputPort("joint_position_desired"),
                station.GetInputPort("iiwa_position"))

teleop = builder.AddSystem(PoseSliders())
teleop.SetRpyRange(RollPitchYaw(0, -0.5, -np.pi), RollPitchYaw(2*np.pi,np.pi, np.pi))
teleop.SetXyzRange([-0.6, -0.8, 0.], [0.8, 0.3, 1.1])
builder.Connect(teleop.get_output_port(0), 
                differential_ik.GetInputPort("rpy_xyz_desired"))
wsg_teleop = builder.AddSystem(SchunkWsgTeleop())
builder.Connect(wsg_teleop.GetOutputPort("position"),
                station.GetInputPort("wsg_position"))
builder.Connect(wsg_teleop.GetOutputPort("force_limit"),
                station.GetInputPort("wsg_force_limit"))

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

station_context = diagram.GetMutableSubsystemContext(
    station, context)

num_iiwa_joints = station.num_iiwa_joints()
station.GetInputPort("iiwa_feedforward_torque").FixValue(
    station_context, np.zeros(num_iiwa_joints))

q0 = station.GetOutputPort("iiwa_position_measured").Eval(
    station_context)
differential_ik.parameters.set_nominal_joint_position(q0)
differential_ik.SetPositions(diagram.GetMutableSubsystemContext(
    differential_ik, simulator.get_mutable_context()), q0)
teleop.SetPose(differential_ik.ForwardKinematics(q0))

simulator.set_target_realtime_rate(1.0)

# Open the meshcat visualizer window (check your pop-up blocker).
open_window(visualizer.vis.url(), "meshcat")

simulator.AdvanceTo(1000.0);

# Teleop Example (2D)

Many of the core concepts in this class can be studied in 2D instead of 3D.  And everything is simpler/cleaner there,  including teleoperation!

In [ ]:
builder = DiagramBuilder()

station = builder.AddSystem(ManipulationStation())
station.SetupPlanarIiwaStation()
station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/"
    + "061_foam_brick.sdf",
    RigidTransform(RotationMatrix.Identity(), [0.6, 0, 0]))
station.Finalize()

visualizer = ConnectMeshcatVisualizer(
    builder, 
    station.get_scene_graph(), 
    station.GetOutputPort("pose_bundle"), 
    zmq_url="new",
    server_args=server_args)
#    jupyter_comms=True)
visualizer.set_planar_viewpoint()

robot = station.get_controller_plant()
params = DifferentialInverseKinematicsParameters(robot.num_positions(),
                                                  robot.num_velocities())

time_step = 0.005
params.set_timestep(time_step)
iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                  iiwa14_velocity_limits))
# These constants are in body frame.
params.set_end_effector_velocity_gain([1, 0, 0, 0, 1, 1])
differential_ik = builder.AddSystem(DifferentialIK(
    robot, robot.GetFrameByName("iiwa_link_7"), params, time_step))
builder.Connect(differential_ik.GetOutputPort("joint_position_desired"),
                station.GetInputPort("iiwa_position"))

teleop = builder.AddSystem(PoseSliders(visible=[True, False, False, True, False, True]))
teleop.SetRpyRange(RollPitchYaw(0, 0, 0), RollPitchYaw(3.4, 0, 0))
teleop.SetXyzRange([-0.6, -0.8, 0.], [0.8, 0.3, 1.1])
builder.Connect(teleop.get_output_port(0), 
                differential_ik.GetInputPort("rpy_xyz_desired"))
wsg_teleop = builder.AddSystem(SchunkWsgTeleop())
builder.Connect(wsg_teleop.GetOutputPort("position"),
                station.GetInputPort("wsg_position"))
builder.Connect(wsg_teleop.GetOutputPort("force_limit"),
                station.GetInputPort("wsg_force_limit"))

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

station_context = diagram.GetMutableSubsystemContext(
    station, context)

num_iiwa_joints = station.num_iiwa_joints()
station.GetInputPort("iiwa_feedforward_torque").FixValue(
    station_context, np.zeros(num_iiwa_joints))

q0 = station.GetOutputPort("iiwa_position_measured").Eval(
    station_context)
differential_ik.parameters.set_nominal_joint_position(q0)
differential_ik.SetPositions(diagram.GetMutableSubsystemContext(
    differential_ik, simulator.get_mutable_context()), q0)
teleop.SetPose(differential_ik.ForwardKinematics(q0))

simulator.set_target_realtime_rate(1.0)

# Open the meshcat visualizer window (check your pop-up blocker).
output.eval_js(f'window.open("{visualizer.vis.url()}", "meshcat");', ignore_result=True)

simulator.AdvanceTo(1000.0);